<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT23/tree/reinisfreibergs-Lab1/Lab1/reinisfreibergs_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: matrix factorization 
**Reinis Freibergs**

# **Abstract**


The objective of the lab is to implement and test the sparse matrix-vector multiplication, QR factorisation with the modified Gram-Schmidt method, directly solving a linear system with QR substitution and backsubstitution, as well as a Least Squares problem. <br>
All algorithms were tested with random matrices with the assumed configuration and returned the same outputs as ready made solutions from numpy within a tolerance of round-off error. 


# **About the code**

In [152]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""


# Author: Reinis Freibergs, 2023

# Based on a template:
# Copyright (C) 2023 Johan Hoffman (jhoffman@kth.se)


# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [180]:
# Load neccessary modules.
#from google.colab import files

import numpy as np
from scipy import sparse

# **Introduction**

Systems of linear equations $Ax=b$ are important in the study of differential and integral equations. In case of the matrix $A$ being nonsingular the solution is given by finding the inverse of $A$:
$$x = A^{-1}b$$

In this report the implementations of important algorithms for finding the solutions of systems of linear equations are given:<br>
1.  Sparse matrix-vector product - for the case when a product needs to be found between a matrix with most elements being zero and a vector.
<br>
2. Modified Gram-Schmidt QR factorization - numerically stable method to factorize a matrix in a product of two easily invertible matrices.
<br>
3. A direct solver with the use of QR factorization and backsubstitution.
4. A least squares problem - for the case of overdetermined matrices

<br>
All implementations are based on materials from the lecture notes.




# **Methods**

### Sparse matrix-vector product

If most components of a matrix are zero it is sparse. In that case a regular dense multiplication involves mostly multiplying by zero and is wasteful. To tackle the problem a specific data structure called _compressed raw storage_ (CRS) can be used, where the matrix is represented with three arrays:
1. non-zero values _val_
1. respective column indices _col_idx_
1. array pointing to the start of each array _row_ptr_

The implementation itself is based on the algorithm 5.9 in lecture notes.


In [163]:
def matrix_product(vector, val, col_idx, row_ptr):
    
    product = np.zeros(shape=len(row_ptr) - 1)
    for i in range(len(row_ptr) - 1):
        for j in range(row_ptr[i], row_ptr[i + 1]):
            product[i] += val[j] * vector[col_idx[j]]
            
    return product

Implementation is tested against the respective _numpy_ function. The sparce matrices are generated randomly and translated in the src format with the scipy package, as it was not an objective of the lab to implement these parts as well.  

In [164]:
for i in range(3):
    columns = np.random.randint(2, 10)
    rows = np.random.randint(2, 10)
    test_matrix = sparse.random(m=rows,
                                n=columns,
                                density=0.1)

    csr_matrix = sparse.csr_matrix(test_matrix)
    test_vector = np.random.rand(columns)
    
    dense_product = test_matrix @ test_vector
    sparse_product = matrix_product(test_vector, csr_matrix.data, csr_matrix.indices, csr_matrix.indptr)
    
    print(f'Test no. {i}, matrix_shape:{rows, columns}, difference: {dense_product - sparse_product}')
    

Test no. 0, matrix_shape:(7, 8), difference: [0. 0. 0. 0. 0. 0. 0.]
Test no. 1, matrix_shape:(6, 6), difference: [0. 0. 0. 0. 0. 0.]
Test no. 2, matrix_shape:(6, 3), difference: [0. 0. 0. 0. 0. 0.]


### QR factorization

QR factorization allows to factor a matrix $A$ into and orthogonal matrix $Q$ and a triangular matrix $R$, where both are easy to invert - for an orthogonal matrix the inverse is given by its transpose $Q^{-1} = Q^T$ and for triangular the inverse can be found by backsubstitution, implemented in the next algorithm.

Exact implementation uses the modified Gram-Schmidt algorithm, based on the algorithm 5.3 from lecture notes, as it's more numerically stable than the classical Gram-Schmidt method.

In [165]:
def modified_gram_schmidt_QR(A):
  n = A.shape[0]
  Q = np.zeros(shape=(n, n))
  R = np.zeros_like(Q)
  for j in range(n):
    v_j = A[:, j]
    for i in range(j):
      R[i, j] = Q[:, i] @ v_j
      v_j = v_j - R[i, j] * Q[:, i]

    R[j, j] = np.linalg.norm(v_j)
    Q[:, j] = v_j / R[j, j]

  return Q, R

For testing the implementation its checked whether $R$ is upper triangular, if $Q^TQ=I$ and $QR=A$, where for the latter two Frobenius norms are used accordingly $\|Q^TQ -I \|$ and $\|QR - A \|$ 

In [191]:
for i in range(3):
    n = np.random.randint(2, 10)
    # add identity matrix to make sure its invertible
    A = np.random.rand(n,n) + np.eye(n)

    Q, R = modified_gram_schmidt_QR(A)
    # R is upper triangular - sum of all indices below diagonal should be 0
    lower_sum = (np.sum(np.tril(R, -1)) == 0)

    f1 = np.linalg.norm(Q.T @ Q - np.eye(n))
    f2 = np.linalg.norm(Q @ R - A)
    
    print(f'test no. {i}')
    print(f'is upper triangular: {lower_sum}') 
    print(f'||Q^TQ -I||: {f1}')
    print(f'||QR -A||: {f2}')



test no. 0
is upper triangular: True
||Q^TQ -I||: 3.9885061540178885e-16
||QR -A||: 3.379462171739212e-16
test no. 1
is upper triangular: True
||Q^TQ -I||: 5.496298777836552e-16
||QR -A||: 4.560714706164002e-16
test no. 2
is upper triangular: True
||Q^TQ -I||: 5.847987538262697e-16
||QR -A||: 3.497780720931476e-16


### Direct solver

The direct solver aims to solve the system of linear equations $Ax=b$ by finding $x = A^{-1}b$. For this the previously implemented QR factorization can be used, as it's resulting matrices are easy to invert. In that case the inversion turns into:
$$Ax = QRx = b$$
$$Rx = Q^{-1}b = Q^Tb$$
where the inverse of matrix $Q^{-1} = Q^{T}$ is given by its transpose as the property of orthogonal matrices, while the inverse of the upper triangular matrix $R$ will be found by implementing the backward substitution algorithm based on 5.2 from the lecture notes.


In [174]:
def backward_substitution(U, b):
    n = len(b)
    x = np.zeros(shape=(n,))
    x[n-1] = b[n-1] / U[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum = 0
        for j in range(i+1, n):
            sum += U[i, j]*x[j]
        x[i] = (b[i] - sum)/U[i, i]
        
    return x

def direct_solver(A, b):
    Q, R = modified_gram_schmidt_QR(A)
    Q_i = Q.T
    Q_i_b = Q_i @ b
    x = backward_substitution(R, Q_i_b)
    
    return x

The implementation is tested with the residuals $\|Ax - b \|$ and $\|x-y \|$ where y is a manufactured solution with $b=Ay$

In [190]:

for i in range(3):
    n = np.random.randint(1,25)
    
    A = np.random.rand(n, n) + np.eye(n)
    b = np.random.rand(n)
    
    x1 = direct_solver(A,b)
    residual_1 = np.linalg.norm(A @ x1 - b)
    
    y = np.random.rand(n)
    b_y = A @ y
    x_y = direct_solver(A, b_y)
    residual_2 = np.linalg.norm(x_y - y)
    
    print(f'test no. {i}')
    print(f'||Ax - b||: {residual_1}')
    print(f'||x - y||: {residual_2}')
    

test no. 0
||Ax - b||: 4.330225622130344e-15
||x - y||: 2.4022911209387004e-13
test no. 1
||Ax - b||: 7.535012591929473e-16
||x - y||: 5.136806496458733e-15
test no. 2
||Ax - b||: 5.36250363036098e-15
||x - y||: 8.925828822937747e-13


### Least squares problem

The least squares problem seeks the best fitting solution $\bar{x}$ for the equation $Ax = b$ for the case of a rectangular $mxn$ matrix with $m>n$, meaning that there are more equations than unknowns, which is often used for data fitting.

The solution can be found by the _pseudoinverse_ or the _Moore-Penrose_ inverse defined in example 2.17

$$A^+ = (A^TA)^{-1}A^T$$

$$\bar{x} = (A^TA)^{-1}A^Tb$$

While using the previously implemented QR factorization to find the inverse of $(A^TA)^{-1}$

In [176]:
def least_squares(A, b):
    a_t_a = A.T @ A
    a_t_b = A.T @ b
    Q, R = modified_gram_schmidt_QR(a_t_a)
    q_t = Q.T
    q_t_a_t_b = q_t @ a_t_b
    x = backward_substitution(R, q_t_a_t_b)
    
    return x
    

Even though the algorithm is tipically used for overdetermined systems $(m>n)$ the tests are going to use both square matrices to test against the residual of exact solution $\|Ax - b \|$ and overdetermined solutions with $m>n$ compared with the numpy solution.

In [187]:
for i in range(3):
    n = np.random.randint(2, 5)
    A = np.random.rand(n, n)
    b = np.random.rand(n)

    x1 = least_squares(A, b)
    x2 = np.linalg.solve(A.T @ A, A.T @ b)
    
    # overdetermined
    m = np.random.randint(n+1, 10)
    A1 = np.random.rand(m, n)
    b1 = np.random.rand(m)   
    
    x1_m = least_squares(A, b)
    x2_m = np.linalg.lstsq(A, b, rcond=None)[0]

    print(f'test no: {i}')
    print(f'||Ax-b|| with m=n: {np.linalg.norm(A@x1-b)}')
    print(f'||Ax-b|| with m>n: {np.linalg.norm(A1@x1_m-b1)}')
    print(f'numpy solution ||Ax - b ||: with m>n: {np.linalg.norm(A1@x2_m-b1)}')

test no: 0
||Ax-b|| with m=n: 9.585647399488852e-13
||Ax-b|| with m>n: 1.855911558098565
numpy solution ||Ax - b ||: with m>n: 1.8559115581019565
test no: 1
||Ax-b|| with m=n: 1.1102230246251565e-16
||Ax-b|| with m>n: 2.1822396537558912
numpy solution ||Ax - b ||: with m>n: 2.1822396537558912
test no: 2
||Ax-b|| with m=n: 3.2850765864159086e-11
||Ax-b|| with m>n: 8.074733577929697
numpy solution ||Ax - b ||: with m>n: 8.074733579402679


# **Results**

All of the algorithms passed the given tests within a tolerance of round-off error.

# **Discussion**


All implemented algorithms both found satisfactory solutions to the given problem and gave the same answers as the default numpy algorithms. For the QR factorization and direct solver only solutions with square matrices were reviewed, which could be further investigated for other shapes. 